<a href="https://colab.research.google.com/github/lalitapatil/Blockchain-Data-Storage/blob/main/SkinCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install split-folders tensorflow

import splitfolders
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Split dataset
splitfolders.ratio('/content/drive/MyDrive/skin_cancer_dataset',
                   output='/content/drive/MyDrive/skin_cancer_split',
                   seed=42,
                   ratio=(.7, .15, .15))

IMAGE_SIZE = [224, 224]

Copying files: 6902 files [16:07,  8.83 files/s]

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

IMAGE_SIZE = [224, 224]

# Load InceptionV3
inception = InceptionV3(input_shape=IMAGE_SIZE + [3],
                        weights='imagenet',
                        include_top=False)

for layer in inception.layers:
    layer.trainable = False

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
# Get number of classes from train folder
from glob import glob
folders = glob('/content/drive/MyDrive/skin_cancer_split/train/*')

In [4]:
folders

['/content/drive/MyDrive/skin_cancer_split/train/begign _keratosis_like',
 '/content/drive/MyDrive/skin_cancer_split/train/melnocytic_nevi',
 '/content/drive/MyDrive/skin_cancer_split/train/melanoma',
 '/content/drive/MyDrive/skin_cancer_split/train/bascal_cell_carcinoma']

In [7]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Build model
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='sigmoid')(x)  # Use softmax for multiclass
model = Model(inputs=inception.input, outputs=prediction)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/skin_cancer_split/train',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

validation_set = val_test_datagen.flow_from_directory(
    '/content/drive/MyDrive/skin_cancer_split/val',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

test_set = val_test_datagen.flow_from_directory(
    '/content/drive/MyDrive/skin_cancer_split/test',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 6471 images belonging to 4 classes.
Found 1386 images belonging to 4 classes.
Found 1388 images belonging to 4 classes.


In [10]:
# Train model
r = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=10,  # Increase epochs
    steps_per_epoch=len(training_set),
    validation_steps=len(validation_set)
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 2616s 6s/step - accuracy: 0.4013 - loss: 5.6804 - val_accuracy: 0.4509 - val_loss: 4.2829
Epoch 2/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 174s 430ms/step - accuracy: 0.4971 - loss: 4.0173 - val_accuracy: 0.5216 - val_loss: 4.2785
Epoch 3/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 170s 420ms/step - accuracy: 0.5126 - loss: 4.6959 - val_accuracy: 0.5036 - val_loss: 4.8877
Epoch 4/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 170s 420ms/step - accuracy: 0.5436 - loss: 4.0182 - val_accuracy: 0.4502 - val_loss: 6.8206
Epoch 5/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 169s 417ms/step - accuracy: 0.5154 - loss: 4.3798 - val_accuracy: 0.4726 - val_loss: 6.3556
Epoch 6/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 170s 420ms/step - accuracy: 0.5521 - loss: 3.8670 - val_accuracy: 0.5087 - val_loss: 5.1609
Epoch 7/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 168s 415ms/step - accuracy: 0.5582 - loss: 3.7754 - val_accuracy: 0.5253 - val_loss: 6.5613
Epoch 8/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 169s 416ms/step - accuracy: 0.5660 - l

In [11]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Accuracy: {test_accuracy:.4f}")

87/87 ━━━━━━━━━━━━━━━━━━━━ 483s 6s/step - accuracy: 0.5218 - loss: 5.2010
Test Accuracy: 0.4885
